<a href="https://colab.research.google.com/github/zgordounc/pollock/blob/main/Front_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install docx2txt
!pip install transformers
!pip install torch
!pip install gradio


In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re
import gradio as gr
import os
import docx2txt


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("allenai/led-base-16384")
model = AutoModelForSeq2SeqLM.from_pretrained("nalmy/pollock-590").to(device)

def summarize(text_file):
    file_extension = os.path.splitext(text_file.name)[1]
    if file_extension == ".txt":
        # Load text from a txt file
        with open(text_file.name, "r", encoding='latin1') as f:
          text = f.read()
    elif file_extension == ".docx":
        # Load text from a Word file
        text = docx2txt.process(text_file.name)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

    #input_ids = tokenizer(text, return_tensors="pt").input_ids
    #global_attention_mask = torch.zeros_like(input_ids)
    # set global_attention_mask on first token
    #global_attention_mask[:, 0] = 1

    inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True)
    inputs = inputs.to(device)
    summary_ids = model.generate(inputs)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


    
    return text, summary


iface = gr.Interface(
    fn=summarize,
    inputs=gr.inputs.File(label="Upload a txt file or a Word file for the input text"),
    outputs=[gr.outputs.Textbox(label="Original text"), gr.outputs.Textbox(label="Summary")],
    title="Academic Paper Summarization Demo",
    description="Upload a txt file or a Word file for the input text. Get a summary generated by a small T5 model from Hugging Face.",
)

iface.launch()

In [ ]:

iface.launch()